## Erreur inverse - Cas symétrique


Soient $A\in \mathcal{S}_n(\mathbb{R})$ inversible, $b\in\mathbb{R}^n$ et $x\in\mathbb{R}^n$ solution de $$Ax=b.$$

On dispose d'un algorithme permettant la résolution numérique d'un tel problème. L'exécution de celui-ci conduit à une solution numérique $\tilde{x}$, entachée d'erreurs : $$A\tilde{x}\neq b. $$

L'erreur inverse de cet algorithme, en norme de Frobenius, peut être obtenue depuis la solution du problème d'optimisation suivant : $$\min_{\Delta \in \mathcal{C}} \Vert \Delta \Vert^2_F,$$

avec $$\mathcal{C}=\left\{ \Delta \in \mathcal{S}_n(\mathbb{R}) \mbox{ t.q. } (A+\Delta)\tilde{x}=b\right\}.$$

On s'intéresse à un problème plus simple qui consiste à chercher un élément de $\mathcal{C}$, si ce dernier n'est pas vide :

$$\hspace{5cm} (\mathcal{E})\quad \mbox{ Trouver }\Delta \in \mathcal{S}_n(\mathbb{R}) \mbox{ t.q. } (A+\Delta)\tilde{x}=b.$$

En écrivant $$\mathcal{C}=\mathcal{C}_1\bigcap\mathcal{C}_2,$$ avec $$\mathcal{C}_1=\mathcal{S}_n(\mathbb{R}), \quad \mathcal{C}_2=\left\{\Delta \in \mathcal{M}_n(\mathbb{R}) \mbox{ t.q. } (A+\Delta)\tilde{x}=b\right\},$$ on est amenés à chercher un élément sur l'intersection de deux convexes fermés non vides.

**Question 1 :** Justifier que ces deux parties sont convexes, fermées et non vides.


On définit pour cela le problème d'optimisation suivant : 
$$\hspace{5cm} (\mathcal{P})\quad \min_{\Delta \in \mathcal{M}_n(\mathbb{R})} f(\Delta)=\max(d(\Delta,\mathcal{C}_1),d(\Delta,\mathcal{C}_2))$$

avec $d(\Delta,\mathcal{C}_i)$ la distance de $\Delta$ à $\mathcal{C}_i$.

**Question 2 :** On choisit de munir $\mathcal{M}_n(\mathbb{R})$ du produit scalaire $<X,Y>=tr(XY^T)$. On a alors $d(\Delta,\mathcal{C}_i)=\Vert \Delta - \Pi_{\mathcal{C}_i}(\Delta)\Vert_F$, avec $\Vert \Vert_F$ la norme de Frobenius, et $\Pi_{\mathcal{C}_i}(\Delta)$ le projeté de $\Delta$ sur $\mathcal{C}_i$. Donner l'expression analytique des $\Pi_{\mathcal{C}_i}(\Delta)$.

**Question 3 :** Proposer un sous-gradient de $f$ en $\Delta$, $\forall \Delta \in \mathcal{C}$.

**Qestion 4 :** Résoudre le problème $ (\mathcal{P})$ par l'algorithme du sous-gradient.


In [1]:
###############################"

using LinearAlgebra
using Plots

n=10

tmp=randn(n,n)
A=0.5*(tmp+transpose(tmp))
A=tmp*transpose(tmp)
x=randn(n,)
tmp=randn(n,n)
var=0.01
Dtrue=var*0.5*(tmp+transpose(tmp))

b=(A+Dtrue)*x

###############################" 

10-element Vector{Float64}:
  -4.078239213630445
   3.839155685805868
 -22.0367261382469
  12.18922555851573
  29.34670852143549
 -12.888224738760345
 -10.070354928172344
  -3.570010230760281
  -5.6692913249117804
 -25.733195780681626

In [2]:
function PiC1(delta)
    return (delta + transpose(delta))/2
end

function PiC2(delta, x, A, b)
    n2 = norm(x)^2
    return delta * (I - (x*transpose(x)) / n2) + (b-A*x)*transpose(x) / n2
end

function evalf(delta, x, A, b)
    diff1 = delta-PiC1(delta)
    n1 = norm(diff1)
    diff2 = delta-PiC2(delta, x, A, b)
    n2 = norm(diff2)
    if n1 > n2
        return n1, diff1 / n1
    else
        return n2, diff2 / n2
    end
end

evalf (generic function with 1 method)

In [3]:
#Initialisation
delta = randn(size(A));
deltaBest=delta;
k = 0;
fbest =1000000; # $f_{best}^0$
histo =[];# Suite des itérés f_{best}^k

lambda=1e-2;

pas=3;
itermax=500;

deltak=delta;
fp, g = evalf(deltak, x, A, b)
while k < itermax
    k = k + 1
    # Insérer votre code
    if pas == 1
        alpha = 1;
    elseif pas == 2
        alpha = 1 / k
    else
        alpha = fp / norm(g)
    end
    deltak = deltak - alpha * g
    fp, g = evalf(deltak, x, A, b)
    if fp < fbest
        fbest = fp
        deltaBest = deltak
    end
    # Fin insérer code
    
    # Stockage
    append!( histo, fbest)
end
#histo
#Affichage des courbes de convergence
plotly();
iter=1:itermax;
plot(iter,histo,title="Convergence curve",label=["f"],lw=3)

[ Info: For saving to png with the Plotly backend PlotlyBase and PlotlyKaleido need to be installed.


In [4]:
RMSE = norm(Dtrue-deltaBest) / norm(Dtrue)

74.96629845951712

In [5]:
ErreurC1 = norm(deltaBest - transpose(deltaBest)) / norm(deltaBest)

1.3967648409081715e-16

In [6]:
ErreurC2 = norm((A+deltaBest)*x-b) / norm(b)

1.1978836607007194e-16

In [7]:
fbest

5.335258313081829e-16